In [56]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from transformers import AutoModel, AutoTokenizer
import torch
from scipy.spatial.distance import cosine
import json
from tqdm import tqdm

In [ ]:
recipe_df = pd.read_csv("kr_recipe.csv")

In [ ]:
# 열 이름 변경
recipe_df.rename(columns={'RCP_SNO': '레시피일련번호', 'RCP_TTL': '레시피제목', 'CKG_NM': '요리명',
                          'RGTR_ID': '등록자ID', 'RGTR_NM': '등록자명', 'INQ_CNT': '조회수',
                          'RCMM_CNT': '추천수', 'SRAP_CNT': '스크랩수', 'CKG_MTH_ACTO_NM': '요리타입',
                          'CKG_STA_ACTO_NM': '음식분위기', 'CKG_MTRL_ACTO_NM': '재료타입', 'CKG_KND_ACTO_NM': '음식타입',
                          'CKG_IPDC': '요리소개', 'CKG_MTRL_CN': '재료리스트', 'CKG_INBUN_NM': '몇인분',
                          'CKG_DODF_NM': '요리난이도', 'CKG_TIME_NM': '요리시간', 'FIRST_REG_DT': '최초등록일시'}, inplace=True)

In [ ]:
# 불필요한 열 제거
recipe_df = recipe_df.drop(columns=['레시피일련번호', '레시피제목', '등록자ID', '등록자명', '요리소개', '최초등록일시'])

# 다음 열들에서 결측치 제거
recipe_df = recipe_df.dropna(subset=['재료리스트', '요리명'])

In [ ]:
# 다음 열들에서 데이터 수정
modify_col_data = ['추천수', '스크랩수']

for col in modify_col_data:
    
    # 숫자로 변환할 수 없는 값 NaN으로 변경
    recipe_df[col] = pd.to_numeric(recipe_df[col], errors='coerce')
    
    # NaN => 0
    recipe_df[col].fillna(0, inplace=True)
    
    # object => int64
    recipe_df[col] = recipe_df[col].astype('int64')

In [17]:
def extraction_of_nouns(sentence):
    # 패턴
    pattern = r'\[\w+\]' # ex) [재료], [양념]

    # 패턴으로 분리
    parts = re.split(pattern, sentence)

    # 각 파트에서 양쪽 여백 제거, 비어있지 않은 리스트만 추출
    parts = [list(map(lambda x: x.strip(), part.split("|"))) for part in parts if part]

    # 2차원 리스트 평탄화
    flattend_parts = [item for part in parts for item in part]

    # Okt 형태소 분석기 로드
    okt = Okt()

    # 명사 추출
    result = []
    for part in flattend_parts:
        nouns = okt.nouns(part)
        # print(f"문장: '{part}' -> 명사: {nouns}")
        result.extend(nouns)
    
    return result

In [25]:
# 진행 상황을 출력하는 헬퍼 함수
def progress_apply(series, func):
    result = []
    for i, item in enumerate(tqdm(series, desc="Processing", unit="row")):
        result.append(func(item))
    return result

In [22]:
# DataFrame에서 '추출된명사' 열 생성
# recipe_df['추출된명사'] = recipe_df['재료리스트'].apply(extraction_of_nouns)
recipe_df['추출된명사'] = progress_apply(recipe_df['재료리스트'], extraction_of_nouns)

Processing: 100%|██████████| 183924/183924 [05:39<00:00, 541.66row/s]


In [57]:
# BERT 모델과 토크나이저 로드
model = AutoModel.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [35]:
# 입력 문장 토크나이징 및 텐서 변환
# def get_word_embedding(word):
#     inputs = tokenizer(word, return_tensors='pt')
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # [CLS] 토큰의 임베딩을 사용
#     embedding = outputs.last_hidden_state[:, 0, :].squeeze()
#     return embedding

def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding

In [59]:
# 라벨 종류 파일 로드
with open('sorted_all_labels.json', 'r', encoding='utf-8') as file:
    all_labels = json.load(file)

In [60]:
# 모든 레이블의 임베딩 미리 계산
label_embeddings = {label: get_word_embedding(label) for label in all_labels if label != "-"}
label_names = list(label_embeddings.keys())
label_matrix = np.array(list(label_embeddings.values()))

In [39]:
# def cosine_similarity(word1, word2):
#     embedding1 = get_word_embedding(word1)
#     embedding2 = get_word_embedding(word2)

#     # 코사인 유사도 계산
#     similarity = 1 - cosine(embedding1.numpy(), embedding2.numpy())
#     # print(f"'{word1}' 와(과) '{word2}' 의 유사도 = {similarity}")
#     return similarity

# def cosine_similarity(embedding1, embedding2):
#     similarity = 1 - cosine(embedding1.numpy(), embedding2.numpy())
#     return similarity

# 코사인 유사도 계산 함수 (벡터화된 방식)
def cosine_similarity_matrix(vectors, label_matrix):
    dot_product = np.dot(vectors, label_matrix.T)
    vector_norms = np.linalg.norm(vectors, axis=1)
    label_norms = np.linalg.norm(label_matrix, axis=1)
    similarity_matrix = dot_product / np.outer(vector_norms, label_norms)
    return similarity_matrix

In [41]:
# 유사한 명사를 찾는 함수
# def find_similar_nouns(nouns, all_labels, threshold=0.7):
#     similar_nouns = []
#     for noun in nouns:
#         for label in all_labels:
#             if label != "-" and cosine_similarity(noun, label) >= threshold:
#                 similar_nouns.append(label)
#     return similar_nouns

# def find_similar_nouns(nouns, label_embeddings, threshold=0.7):
#     similar_nouns = []
#     for noun in nouns:
#         noun_embedding = get_word_embedding(noun)
#         for label, label_embedding in label_embeddings.items():
#             if cosine_similarity(noun_embedding, label_embedding) >= threshold:
#                 similar_nouns.append(label)
#     return similar_nouns

def find_similar_nouns(nouns, label_matrix, label_names, threshold=0.7):
    if not nouns:
        return []
    noun_embeddings = np.array([get_word_embedding(noun) for noun in nouns])
    similarity_matrix = cosine_similarity_matrix(noun_embeddings, label_matrix)
    similar_nouns = set()
    for idx, noun in enumerate(nouns):
        similarities = similarity_matrix[idx]
        similar_indices = np.where(similarities >= threshold)[0]
        similar_nouns.update([label_names[i] for i in similar_indices])
    return list(similar_nouns)

In [65]:
# '유사한재료' 열 생성
# recipe_df['유사한재료'] = recipe_df['추출된명사'].apply(lambda x: find_similar_nouns(x, all_labels))
# recipe_df['유사한재료'] = progress_apply(recipe_df['추출된명사'], lambda x: find_similar_nouns(x, all_labels))
# recipe_df['유사한재료'] = progress_apply(recipe_df['추출된명사'], lambda x: find_similar_nouns(x, label_embeddings))
df = recipe_df.iloc[:100]
df['유사한재료'] = progress_apply(recipe_df.iloc[:100]['추출된명사'], lambda x: find_similar_nouns(x, label_matrix, label_names))
# recipe_df['유사한재료'] = progress_apply(recipe_df['추출된명사'], lambda x: find_similar_nouns(x, label_matrix, label_names))

Processing: 100%|██████████| 100/100 [00:53<00:00,  1.86row/s]
C:\Users\Seunghwan\AppData\Local\Temp\ipykernel_33520\907909059.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['유사한재료'] = progress_apply(recipe_df.iloc[:100]['추출된명사'], lambda x: find_similar_nouns(x, label_matrix, label_names))


In [68]:
pd.set_option('display.max_colwidth', None)

df[['재료리스트', '추출된명사', '유사한재료']]

,재료리스트,추출된명사,유사한재료
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T,"[어묵, 개, 김밥, 용김, 장, 당면, 움큼, 양파, 개, 당근, 개, 깻잎, 장, 튀김, 가루, 컵, 올리브유, 적당, 량, 간장, 참기름]","[여주, 파르메산 치즈, 크림치즈, 백미, 모짜렐라 치즈, 청경채, 돼지 삼겹살, 멜론, 감자, 카레 가루, 호두, 생강, 주키니, 콜리플라워, 콩, 파인애플, 홍고추, 해쉬 브라운, 나초 칩, 번, 파파야, 게살, 타이 그린 칠리, 버섯, 소시지, 양파, 퀘크, 그린 샐러드, 머스터드, 복숭아, 패션후르츠, 통조림 옥수수, 시금치, 귤, 쿠키, 우유, 주먹밥, 갈비, 푸딩, 고추가루, 휘핑크림, 석류, 치즈, 참기름, 견과류, 마요네즈, 오리, 참깨, 호박, 잼, 토마토, 그린 애플, 사워크림, 수세미오이, 오렌지, 코코넛, 가루, 병, 초콜릿 빵, 샐러드, 생선, 피멘토, 자두, 브뤼셀 새싹, 케이크 롤, 만두, 닭 넓적다리, 아티초크, 캔, 모렐 버섯, 초콜릿, 가재, 도시락, 고구마, 간장, 올리브, 수박, 가지, 셀러리, 후추, 햄, 돼지고기, 상추, 블루베리, 통조림 참치, 오레오, 단무지, 고수, 차, 아스파라거스, 고추, 옥수수, 케이크, 타임, 회향, 바나나, 갈린 치즈, 요거트, 오크라, 자몽, ...]"
1,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개,"[두부, 모, 당근, 개, 고추, 개, 브로콜리, 개, 새우, 마리, 녹말가루, 계란, 개]","[여주, 파르메산 치즈, 크림치즈, 백미, 모짜렐라 치즈, 청경채, 돼지 삼겹살, 멜론, 감자, 카레 가루, 호두, 생강, 주키니, 콜리플라워, 콩, 파인애플, 홍고추, 해쉬 브라운, 나초 칩, 번, 파파야, 게살, 타이 그린 칠리, 버섯, 소시지, 양파, 퀘크, 그린 샐러드, 머스터드, 복숭아, 패션후르츠, 통조림 옥수수, 시금치, 귤, 쿠키, 우유, 주먹밥, 갈비, 푸딩, 고추가루, 휘핑크림, 석류, 치즈, 참기름, 견과류, 마요네즈, 오리, 참깨, 호박, 잼, 토마토, 그린 애플, 사워크림, 수세미오이, 오렌지, 코코넛, 가루, 병, 초콜릿 빵, 샐러드, 생선, 피멘토, 자두, 브뤼셀 새싹, 케이크 롤, 만두, 닭 넓적다리, 아티초크, 캔, 모렐 버섯, 초콜릿, 가재, 도시락, 고구마, 간장, 올리브, 수박, 가지, 셀러리, 후추, 햄, 돼지고기, 상추, 블루베리, 통조림 참치, 오레오, 단무지, 고수, 차, 아스파라거스, 고추, 옥수수, 케이크, 타임, 회향, 바나나, 갈린 치즈, 요거트, 오크라, 자몽, ...]"
2,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간| 날치알 6스푼| 김가루 약간| 후리가케(또는밥이랑같은류)| 참기름 약간| 통깨 약간| 계란 노른자 2알,"[밥, 공기, 당근, 개, 치자, 단무지, 개, 김치, 쪽, 무순, 약간, 날치, 알, 스푼, 김, 가루, 약간, 후리, 케, 밥, 류, 참기름, 약간, 통깨, 약간, 계란, 노른자, 알]","[여주, 파르메산 치즈, 크림치즈, 백미, 모짜렐라 치즈, 청경채, 돼지 삼겹살, 멜론, 감자, 카레 가루, 호두, 생강, 주키니, 콜리플라워, 콩, 파인애플, 홍고추, 해쉬 브라운, 나초 칩, 번, 파파야, 게살, 타이 그린 칠리, 버섯, 소시지, 양파, 퀘크, 그린 샐러드, 머스터드, 복숭아, 패션후르츠, 통조림 옥수수, 시금치, 귤, 쿠키, 우유, 주먹밥, 갈비, 푸딩, 고추가루, 휘핑크림, 석류, 치즈, 참기름, 견과류, 마요네즈, 오리, 참깨, 호박, 잼, 토마토, 사워크림, 수세미오이, 코코넛, 오렌지, 그린 애플, 가루, 병, 초콜릿 빵, 샐러드, 생선, 피멘토, 자두, 브뤼셀 새싹, 케이크 롤, 만두, 닭 넓적다리, 아티초크, 캔, 모렐 버섯, 초콜릿, 가재, 도시락, 고구마, 간장, 올리브, 수박, 가지, 셀러리, 후추, 햄, 돼지고기, 상추, 블루베리, 통조림 참치, 오레오, 단무지, 고수, 차, 아스파라거스, 고추, 옥수수, 케이크, 타임, 회향, 바나나, 갈린 치즈, 요거트, 오크라, 자몽, ...]"
3,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 컵, 찹쌀, 컵, 호두, 물, 컵, 소금, 약간]","[여주, 파르메산 치즈, 크림치즈, 백미, 모짜렐라 치즈, 청경채, 돼지 삼겹살, 멜론, 감자, 카레 가루, 호두, 생강, 콜리플라워, 콩, 파인애플, 홍고추, 나초 칩, 번, 파파야, 게살, 타이 그린 칠리, 버섯, 소시지, 양파, 퀘크, 그린 샐러드, 머스터드, 복숭아, 통조림 옥수수, 시금치, 귤, 쿠키, 우유, 주먹밥, 갈비, 푸딩, 고추가루, 휘핑크림, 석류, 치즈, 참기름, 견과류, 마요네즈, 오리, 참깨, 호박, 잼, 토마토, 코코넛, 수세미오이, 그린 애플, 오렌지, 사워크림, 가루, 병, 초콜릿 빵, 샐러드, 생선, 피멘토, 자두, 브뤼셀 새싹, 케이크 롤, 만두, 닭 넓적다리, 아티초크, 캔, 모렐 버섯, 초콜릿, 가재, 도시락, 고구마, 간장, 올리브, 수박, 가지, 셀러리, 후추, 햄, 돼지고기, 상추, 블루베리, 통조림 참치, 오레오, 단무지, 고수, 차, 아스파라거스, 고추, 옥수수, 케이크, 회향, 타임, 바나나, 갈린 치즈, 요거트, 오크라, 자몽, 케첩, 염소 치즈, 누텔라, ...]"
4,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간,"[북어, 포, 마리, 찹쌀, 가루, 간장, 설탕, 물, 파, 마늘, 참기름, 깨소금, 후춧가루, 약간]","[여주, 파르메산 치즈, 크림치즈, 백미, 모짜렐라 치즈, 청경채, 돼지 삼겹살, 멜론, 감자, 카레 가루, 호두, 생강, 주키니, 콜리플라워, 콩, 파인애플, 홍고추, 나초 칩, 번, 파파야, 게살, 타이 그린 칠리, 버섯, 소시지, 양파, 퀘크, 그린 샐러드, 머스터드, 복숭아, 패션후르츠, 통조림 옥수수, 시금치, 귤, 쿠키, 우유, 주먹밥, 갈비, 푸딩, 고추가루, 휘핑크림, 석류, 치즈, 참기름, 호박, 오리, 참깨, 견과류, 마요네즈, 잼, 토마토, 그린 애플, 사워크림, 수세미오이, 오렌지, 코코넛, 가루, 병, 초콜릿 빵, 샐러드, 생선, 피멘토, 자두, 브뤼셀 새싹, 케이크 롤, 만두, 닭 넓적다리, 아티초크, 캔, 모렐 버섯, 초콜릿, 가재, 도시락, 고구마, 간장, 올리브, 수박, 가지, 셀러리, 후추, 햄, 돼지고기, 상추, 블루베리, 통조림 참치, 오레오, 단무지, 고수, 차, 아스파라거스, 고추, 옥수수, 케이크, 타임, 회향, 바나나, 갈린 치즈, 요거트, 오크라, 자몽, 누텔라, ...]"
...,...,...,...
95,[재료] 다시마| 가다랑이포| 모밀면| 김| 대파| 유부 [양념] 간장| 미림,"[다시마, 이포, 모밀, 김, 대파, 유부, 간장, 미림]","[여주, 파르메산 치즈, 크림치즈, 백미, 모짜렐라 치즈, 청경채, 돼지 삼겹살, 멜론, 감자, 카레 가루, 호두, 생강, 주키니, 콜리플라워, 콩, 파인애플, 홍고추, 해쉬 브라운, 나초 칩, 번, 파파야, 게살, 타이 그린 칠리, 버섯, 소시지, 양파, 퀘크, 그린 샐러드, 머스터드, 복숭아, 패션후르츠, 통조림 옥수수, 시금치, 귤, 쿠키, 우유, 주먹밥, 갈비, 푸딩, 고추가루, 휘핑크림, 석류, 치즈, 참기름, 견과류, 마요네즈, 오리, 참깨, 호박, 잼, 토마토, 그린 애플, 사워크림, 수세미오이, 오렌지, 코코넛, 가루, 병, 초콜릿 빵, 샐러드, 생선, 피멘토, 자두, 브뤼셀 새싹, 케이크 롤, 만두, 닭 넓적다리, 아티초크, 캔, 모렐 버섯, 초콜릿, 가재, 도시락, 고구마, 간장, 올리브, 수박, 가지, 셀러리, 후추, 햄, 돼지고기, 상추, 블루베리, 통조림 참치, 오레오, 단무지, 고수, 차, 아스파라거스, 고추, 옥수수, 케이크, 회향, 타임, 바나나, 갈린 치즈, 요거트, 오크라, 자몽, ...]"
96

In [69]:
recipe_df

,요리명,조회수,추천수,스크랩수,요리타입,음식분위기,재료타입,음식타입,재료리스트,몇인분,요리난이도,요리시간,추출된명사
0,어묵김말이,10072,6,66,튀김,간식,가공식품류,디저트,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T,2인분,초급,60분이내,"[어묵, 개, 김밥, 용김, 장, 당면, 움큼, 양파, 개, 당근, 개, 깻잎, 장, 튀김, 가루, 컵, 올리브유, 적당, 량, 간장, 참기름]"
1,두부새우전,5817,3,27,부침,일상,해물류,밑반찬,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개,3인분,초급,30분이내,"[두부, 모, 당근, 개, 고추, 개, 브로콜리, 개, 새우, 마리, 녹말가루, 계란, 개]"
2,알밥,6975,8,36,굽기,일상,해물류,밥/죽/떡,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간| 날치알 6스푼| 김가루 약간| 후리가케(또는밥이랑같은류)| 참기름 약간| 통깨 약간| 계란 노른자 2알,2인분,초급,30분이내,"[밥, 공기, 당근, 개, 치자, 단무지, 개, 김치, 쪽, 무순, 약간, 날치, 알, 스푼, 김, 가루, 약간, 후리, 케, 밥, 류, 참기름, 약간, 통깨, 약간, 계란, 노른자, 알]"
3,현미호두죽,3339,0,11,끓이기,일상,쌀,밥/죽/떡,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,"[현미, 컵, 찹쌀, 컵, 호두, 물, 컵, 소금, 약간]"
4,북어갈비,7173,3,97,굽기,술안주,건어물류,메인반찬,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간,2인분,초급,60분이내,"[북어, 포, 마리, 찹쌀, 가루, 간장, 설탕, 물, 파, 마늘, 참기름, 깨소금, 후춧가루, 약간]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183919,명란젓크림스파게티,59,0,0,기타,손님접대,해물류,양식,[재료] 올리브오일| 스파게티면 80g| 명란젓 1덩이| 생크림 100g| 마늘 1~2톨| 양파 1/4개| 소금 약간| 후추 약간,1인분,아무나,NaN,"[올리브오일, 스파게티, 명란젓, 덩이, 생크림, 마늘, 톨, 양파, 개, 소금, 약간, 후추, 약간]"
183920,아마트리치아나스파게티,46,0,0,기타,손님접대,돼지고기,양식,[재료] 올리브오일| 스파게티면 280~300g| 관찰레 120g| 홀토마토 1캔| 화이트와인 50ml| 파르미지아노 치즈 100g| 소금 약간,4인분,아무나,30분이내,"[올리브오일, 스파게티, 관찰, 레, 홀, 토마토, 캔, 화이트와인, 파르, 미지, 치즈, 소금, 약간]"
183921,오징어젓무침,110,0,8,무침,일상,가공식품류,김치/젓갈/장류,[재료] 오징어젓200g| 무말랭이15g| 청양고추2개| 홍고추1개| 오이고추1개| 쪽파 2줄| 양파 1/4개| 다진마늘 1T| 통깨 1T| 참기름 1T,2인분,아무나,NaN,"[오징어, 젓, 무, 말랭이, 청양고추, 개, 홍, 고추, 개, 오이, 고추, 개, 쪽파, 줄, 양파, 개, 마늘, 통깨, 참기름]"
183922,계란찜,170,0,6,찜,일상,달걀/유제품,밑반찬,[재료] 달걀 6개| 우유 200ml| 물 100ml| 맛살 3개| 당근 1/3개| 대파 1/3대 [양념] 소금 1/3| 참치액 1/2| 맛술 1/2,3인분,아무나,30분이내,"[달걀, 개, 우유, 물, 맛살, 개, 당근, 개, 대파, 소금, 참치, 액, 맛술]"
